#### Curvature Calculation

In [42]:
import numpy as np
import sys

sys.path.insert(0, ".")

In [36]:
def radius_center(A, B, C):

    # A = np.array([2.0, 1.5, 0.0])
    # B = np.array([6.0, 4.5, 0.0])
    # C = np.array([11.75, 6.25, 0.0])
    a = np.linalg.norm(C - B)
    b = np.linalg.norm(C - A)
    c = np.linalg.norm(B - A)
    s = (a + b + c) / 2
    radius = a*b*c / 4 / np.sqrt(s * (s - a) * (s - b) * (s - c))
    b1 = a*a * (b*b + c*c - a*a)
    b2 = b*b * (a*a + c*c - b*b)
    b3 = c*c * (a*a + b*b - c*c)
    center = np.column_stack((A, B, C)).dot(np.hstack((b1, b2, b3)))
    center /= b1 + b2 + b3

    return (radius, center) if radius > 0 else (np.inf, np.array([np.inf, np.inf, np.inf]))


# print(R)
# print(P)

# 15.899002930062531
# array([ 13.42073171,  -9.56097561,   0.        ])

In [8]:
from mediapipe.tasks.python.vision.hand_landmarker import HandLandmarkerResult
from mediapipe.tasks.python.components.containers.category import Category
from mediapipe.tasks.python.components.containers.landmark import NormalizedLandmark, Landmark

results = HandLandmarkerResult(
    handedness=[[Category(index=1, score=0.9926292896270752, display_name="Left", category_name="Left")]],
    hand_landmarks=[
        [
            NormalizedLandmark(x=0.6419628858566284, y=0.616515040397644, z=-6.035641604285047e-07, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5716654062271118, y=0.6108736395835876, z=-0.002957998076453805, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5272430777549744, y=0.5887590646743774, z=-0.01388684194535017, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4818960428237915, y=0.5766850113868713, z=-0.029851868748664856, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.44992589950561523, y=0.5541561841964722, z=-0.04740559309720993, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6141445636749268, y=0.43760523200035095, z=0.007367989979684353, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5445273518562317, y=0.428905725479126, z=-0.007748082280158997, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5036388635635376, y=0.45918241143226624, z=-0.025927452370524406, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.47835105657577515, y=0.49549776315689087, z=-0.03698492795228958, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6072747707366943, y=0.42586055397987366, z=-0.00027601292822510004, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5201941132545471, y=0.42209744453430176, z=-0.009670299477875233, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4791445732116699, y=0.4669061303138733, z=-0.02469821274280548, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.46002593636512756, y=0.5104443430900574, z=-0.03566863387823105, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5915679335594177, y=0.4281168580055237, z=-0.009448129683732986, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5056634545326233, y=0.4338810443878174, z=-0.017011014744639397, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4680931866168976, y=0.4771495461463928, z=-0.02728145755827427, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4502561092376709, y=0.516810953617096, z=-0.034487128257751465, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5680385231971741, y=0.4449925422668457, z=-0.019567394629120827, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.504776656627655, y=0.45213910937309265, z=-0.022505883127450943, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.47540536522865295, y=0.47587350010871887, z=-0.023087508976459503, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4563266634941101, y=0.5018525123596191, z=-0.022691285237669945, visibility=0.0, presence=0.0),
        ]
    ],
    hand_world_landmarks=[
        [
            Landmark(x=0.009037008509039879, y=0.06189635396003723, z=-0.03779924288392067, visibility=0.0, presence=0.0),
            Landmark(x=-0.0033948568161576986, y=0.05620351806282997, z=-0.012544753029942513, visibility=0.0, presence=0.0),
            Landmark(x=-0.02560390532016754, y=0.04868587106466293, z=0.014155255630612373, visibility=0.0, presence=0.0),
            Landmark(x=-0.04892528057098389, y=0.04613838344812393, z=0.05510091409087181, visibility=0.0, presence=0.0),
            Landmark(x=-0.06683654338121414, y=0.034338757395744324, z=0.08791903406381607, visibility=0.0, presence=0.0),
            Landmark(x=-0.0019140357617288828, y=0.000755488988943398, z=0.01971326768398285, visibility=0.0, presence=0.0),
            Landmark(x=-0.021188266575336456, y=-0.0017718548187986016, z=0.027694974094629288, visibility=0.0, presence=0.0),
            Landmark(x=-0.037010885775089264, y=0.001074992585927248, z=0.03401287645101547, visibility=0.0, presence=0.0),
            Landmark(x=-0.05434488505125046, y=0.01770664006471634, z=0.030251258984208107, visibility=0.0, presence=0.0),
            Landmark(x=0.00110423075966537, y=-0.003265355248004198, z=0.007934767752885818, visibility=0.0, presence=0.0),
            Landmark(x=-0.028509020805358887, y=-0.013048581779003143, z=0.018052339553833008, visibility=0.0, presence=0.0),
            Landmark(x=-0.0526660792529583, y=0.0021621340420097113, z=0.023894505575299263, visibility=0.0, presence=0.0),
            Landmark(x=-0.06124870851635933, y=0.01557418704032898, z=0.03212356194853783, visibility=0.0, presence=0.0),
            Landmark(x=-0.0001015738962450996, y=-0.0032526326831430197, z=-0.01480625756084919, visibility=0.0, presence=0.0),
            Landmark(x=-0.029653087258338928, y=-0.0027253220323473215, z=0.0009754223865456879, visibility=0.0, presence=0.0),
            Landmark(x=-0.05461142584681511, y=0.00921244453638792, z=0.011858618818223476, visibility=0.0, presence=0.0),
            Landmark(x=-0.0628863126039505, y=0.022142725065350533, z=0.02059471420943737, visibility=0.0, presence=0.0),
            Landmark(x=-0.01317686215043068, y=0.010379736311733723, z=-0.027534576132893562, visibility=0.0, presence=0.0),
            Landmark(x=-0.02998003363609314, y=0.0076247043907642365, z=-0.01695789210498333, visibility=0.0, presence=0.0),
            Landmark(x=-0.049735985696315765, y=0.005921175237745047, z=-0.008792607113718987, visibility=0.0, presence=0.0),
            Landmark(x=-0.05839437246322632, y=0.01687566563487053, z=4.301162698538974e-05, visibility=0.0, presence=0.0),
        ]
    ],
)

In [14]:
landmarks = np.array(results.hand_landmarks[0])

In [13]:
def to_ndarray(landmark):
    return np.array([landmark.x, landmark.y, landmark.z])

In [15]:
to_ndarray(landmarks[0])

array([ 6.41962886e-01,  6.16515040e-01, -6.03564160e-07])

In [41]:
r, c = radius_center(
    to_ndarray(landmarks[0]),
    to_ndarray(landmarks[1]),
    to_ndarray(landmarks[4]),
)
r

0.24047688819905672

In [43]:
landmarks[[0, 1, 2]]

array([NormalizedLandmark(x=0.6419628858566284, y=0.616515040397644, z=-6.035641604285047e-07, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5716654062271118, y=0.6108736395835876, z=-0.002957998076453805, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5272430777549744, y=0.5887590646743774, z=-0.01388684194535017, visibility=0.0, presence=0.0)],
      dtype=object)

In [34]:
r == 0

False

In [27]:
if _[0]:
    print(f"True")
else:
    print(f"False")

True
